In [19]:
from FinMind.data import DataLoader 
import pandas as pd



In [61]:
## Get data
stock_no = '00919'
dl = DataLoader()
stock_data = dl.taiwan_stock_daily(
    stock_id=stock_no, 
    start_date='2012-09-30',
    end_date='2024-09-30')
stock_data.to_csv('stock_data.csv', index=False)
print(stock_data.keys())


2024-09-27 00:37:34.032 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPrice, data_id: 00919


Index(['date', 'stock_id', 'Trading_Volume', 'Trading_money', 'open', 'max',
       'min', 'close', 'spread', 'Trading_turnover'],
      dtype='object')


In [55]:
## calculate MA
df = pd.read_csv('stock_data.csv')

def calculate_ma(data, window):
    return data.rolling(window=window).mean()
    
ma_window = 60
df[f'MA_{ma_window}'] = df['close'].rolling(window=ma_window).mean()
df.to_csv('stock_data.csv', index=False)

print(df[[f'MA_{ma_window}', 'close']].tail())




         MA_60  close
464  24.732000  24.18
465  24.694000  23.61
466  24.659667  23.74
467  24.628000  23.95
468  24.592833  24.03


In [57]:
## 找出 均線黃金交叉 與 均線死亡交叉, 以 50 與 200 為例
df['MA_200'] = df['close'].rolling(window=200).mean()
df['MA_60'] = df['close'].rolling(window=60).mean()

df['is_golden_cross'] = (df['MA_60'].shift(1) < df['MA_200'].shift(1)) & (df['MA_60'] > df['MA_200'])
df['is_death_cross'] = (df['MA_60'].shift(1) > df['MA_200'].shift(1)) & (df['MA_60'] < df['MA_200'])

print(f'黃金交叉:\n{df[df['is_golden_cross']]['date']} \n')
print(f'死亡交叉:\n{df[df['is_death_cross']]['date']} \n')

黃金交叉:
Series([], Name: date, dtype: object) 

死亡交叉:
Series([], Name: date, dtype: object) 



In [58]:
## 買入信號：前一天的關盤價 < 前一天的 MA 且 當天的關盤價 > 當天的 MA
df['is_buy_signal'] = (df['close'].shift(1) < df[f'MA_{ma_window}'].shift(1)) & (df['close'] > df[f'MA_{ma_window}']) 


In [59]:
## 賣出信號: 前一天的關盤價 > 前一天的 MA 且 當天的關盤價 < 當天的 MA
df['is_sell_signal'] = (df['close'].shift(1) > df[f'MA_{ma_window}'].shift(1)) & (df['close'] < df[f'MA_{ma_window}']) 


In [60]:
# 找出所有買入信號的日期
buy_dates = df[df['is_buy_signal']]['date']

# 找出所有賣出信號的日期
sell_dates = df[df['is_sell_signal']]['date']

print("買入信號日期：")
print(buy_dates.tail())

print("\n賣出信號日期：")
print(sell_dates.tail())

買入信號日期：
137    2023-05-22
249    2023-11-02
362    2024-04-24
364    2024-04-26
Name: date, dtype: object

賣出信號日期：
119    2023-04-25
236    2023-10-16
359    2024-04-19
363    2024-04-25
421    2024-07-18
Name: date, dtype: object
